# Data Preparation, Pipelines & Model 

In [ ]:
# Modules importeren
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Dataset importeren 
df = pd.read_csv("/Users/odessa/Desktop/Applied Data Science & AI/Data Science/Code Inleiding data science/song_data.csv")

In [ ]:
# Target variabele maken 
target = 'song_popularity'

In [ ]:
#df.drop(columns=["song_name"], inplace=True) # inplace=True veranderd de originele dataframe zonder nieuwe dataframe te maken 

### Phase 3: Data Preparation

In [2]:
# 2 nummers droppen
df = df.drop([7119, 11171]).reset_index(drop=True)

In [3]:
print(f"Totaal aantal waardes in de dataframe vóór het verwijderen van dubbele waardes uit song_name en song_duration_ms: {len(df)}")

# Dubbele waardes droppen van song_name en song_duration 
# Als ik alleen song_name duplicates zou verwijderen, zou ik misschien covers van nummers verwijderen, dus daarom check ik ook de song_duration 
df.drop_duplicates(subset=['song_name', 'song_duration_ms'], inplace = True)
print(f"Totaal aantal waardes in de dataframe na verwijderen van dubbele waardes uit song_name en song_duration_ms: {len(df)}")

Totaal aantal waardes in de dataframe vóór het verwijderen van dubbele waardes uit song_name en song_duration_ms: 18833
Totaal aantal waardes in de dataframe na verwijderen van dubbele waardes uit song_name en song_duration_ms: 14466


In [4]:
df.drop(columns=["song_name"], inplace=True) # inplace=True veranderd de originele dataframe zonder nieuwe dataframe te maken 

In [5]:
X = df.drop(columns=[target], axis=1)
y = df[target]

## Winsorizer Class

In [6]:
# BaseEstimator zorgt dat sklearn mijn class kan herkennen als model/stap in pipeline.
# TransformerMixin geeft .fit_transform().
class Winsorizer(BaseEstimator, TransformerMixin):
    def __init__(self, kolommen): 
        self.kolommen = kolommen 
        self.grenzen_ = None # '_' betekent dat het attribuut pas beschikbaar wordt, nadat fit() is uitgevoerd. 
                             # None, omdat de grenzen nog niet bestaan -- worden berekend bij fit().

    def fit(self, X, y=None):
        """Bereken de onder- en bovengrenzen per kolom met interkwartielafstand-regel."""
        self.grenzen_ = {}
        for kolom in self.kolommen:
            Q1 = X[kolom].quantile(0.25)
            Q3 = X[kolom].quantile(0.75)
            IKR = Q3 - Q1 
            ondergrens = Q1 - 1.5 * IKR
            bovengrens = Q3 + 1.5 * IKR
            self.grenzen_[kolom] = (ondergrens, bovengrens)
        return self 
    
    def transform(self, X):
        """Winsoriseer uitschieters: vervang alle waardes buiten de grenzen met de dichtstbijzijnde grenswaarde."""
        X = X.copy() # Kopie maken van data 
        for kolom, (ondergrens, bovengrens) in self.grenzen_.items():
            X.loc[X[kolom] < ondergrens, kolom] = ondergrens 
            X.loc[X[kolom] > bovengrens, kolom] = bovengrens 
        return X
    def get_feature_names_out(self, input_features=None):
        return np.array(self.kolommen)

## Key Cyclic Encoder class 

In [7]:
class ToonsoortCyclischeEncoder(BaseEstimator, TransformerMixin):
    """
    Cyclisch encoden van de toonsoort/key (0-11) met sinus en cosinus. 
    """

    def __init__(self, kolom='key', max_waarde=12):
        self.kolom = kolom
        self.max_waarde = max_waarde
    
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X):
        X = X.copy()
        k = X[self.kolom]
        X[f'{self.kolom}_sin'] = np.sin(2 * np.pi * k/self.max_waarde)
        X[f'{self.kolom}_cos'] = np.cos(2 * np.pi * k/self.max_waarde)
        return X.drop(columns=[self.kolom])
    def get_feature_names_out(self, input_features=None):
        return np.array(self.kolommen)

### Phase 4: Modeling 

Supervised learning, omdat je de uitkomst al hebt 
<br>
Supervised learning heeft 2 hoofdtakken: regressie en classificatie 
<br>
RMSE 
<br>
Meervoudige lineare regressie 
<br>
Logistieke lineare regressie is classification 
<br>
Random forests is het begin van dat machine learning slim werd 

In [8]:
def nieuwe_features(X):
    X = X.copy()
    X['energy_dance'] = X['energy'] * X['danceability']
    X['tempo_loudness'] = X['tempo'] * X['loudness']
    X['valence_energy'] = X['audio_valence'] * X['energy']
    X['acoustic_energy_ratio'] = X['acousticness'] / (X['energy'] + 0.001)
    X['speech_loudness_ratio'] = X['speechiness'] / (abs(X['loudness']) + 0.001)
    X['acoustic_dance_ratio'] = X['acousticness'] / (X['danceability'] + 0.001)
    X['energy_per_tempo'] = X['energy'] / (X['tempo'] + 0.001)
    X['valence_dance'] = X['audio_valence'] * X['danceability']
    X['duration_energy_ratio'] = X['song_duration_ms'] / (X['energy'] + 0.001)

    # Niet - lineaire verbanden 
    X['energy_sq'] = X['energy'] ** 2
    X['tempo_log'] = np.log1p(X['tempo'])
    X['loudness_sq'] = X['loudness'] ** 2
    return X
feature_engineering = FunctionTransformer(nieuwe_features, validate=False)

In [9]:
# Train en test set maken 
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.2, random_state = 42
)

In [10]:
# Kolommen indelen
kolommen_winsoriseren = ['song_duration_ms', 'loudness', 'tempo']
categorische_kolommen = ['time_signature']
cyclische_kolommen = ['key']

#Alle nummerieke kolommen behalve target
numerieke_kolommen = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
overige_kolommen = [
    c for c in numerieke_kolommen 
    if c not in kolommen_winsoriseren + categorische_kolommen + cyclische_kolommen
]

## Pipelines 

In [11]:
# Preprocessing
preprocessor = ColumnTransformer([
    ('winsor_scale', Pipeline ([ # Pipeline zorgt dat alle transformaties in de serie worden uitgevoerd op dezelfde kolommen. 
        ('winsor', Winsorizer(kolommen=kolommen_winsoriseren)),
        ('scaler', StandardScaler()) # Dit moet hier in winsor_scale, omdat anders de gewinsoriseerde kolommen weer weggegooid worden. 
    ]), kolommen_winsoriseren),
    
    ('onehot', OneHotEncoder(handle_unknown='ignore'), categorische_kolommen),
    
    ('key_cyclisch', Pipeline([
        ('encoder', ToonsoortCyclischeEncoder(kolom='key', max_waarde=12)),
        ('scaler', StandardScaler())
    ]), cyclische_kolommen),
    ('scale_overige_kolommen', StandardScaler(), overige_kolommen),
],
remainder=StandardScaler()) # Anders vallen nieuwe features weg 

In [12]:
# Lineaire Regressie 
lineair_pipeline = Pipeline([
    ('feature_creation', feature_engineering),
    ('preprocess', preprocessor),
    ('model', LinearRegression())
])

In [13]:
# Trainen en evalueren 
lineair_pipeline.fit(X_train, y_train)
y_pred = lineair_pipeline.predict(X_test)

In [14]:
random_forest_pipeline = Pipeline([
    ('feature_creation', feature_engineering),
    ('preprocess', preprocessor),
    ('model', RandomForestRegressor(
        random_state=42,
        n_jobs=-1
    ))
])

In [ ]:
param_grid = {
    'model__n_estimators': [100, 200, 400],
    'model__max_depth': [6, 8, 10, 12, 14],
    'model__min_samples_split': [2, 4, 6, 8],
    'model__min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(
    estimator=random_forest_pipeline,
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)


In [16]:
beste_model = grid_search.best_estimator_
y_pred_rf = beste_model.predict(X_test)

r2_rf = r2_score(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = root_mean_squared_error(y_test, y_pred_rf)

# Evaluatie
print("Beste parameters:", grid_search.best_params_)
print(f"Cross-val (train) R²: {grid_search.best_score_:.4f}")
print(f"Test R²: {r2_rf:.4f}")
print(f"Test RMSE: {rmse_rf:.4f}")

Beste parameters: {'model__max_depth': 10, 'model__min_samples_leaf': 1, 'model__min_samples_split': 6, 'model__n_estimators': 400}
Cross-val (train) R²: 0.0580
Test R²: 0.0520
Test RMSE: 20.0811


In [19]:
resultaten = pd.DataFrame({
    'Model': ['Lineaire regressie', 'Random Forest'],
    'R2': [r2_score(y_test, y_pred), r2_rf],
    'RMSE': [root_mean_squared_error(y_test, y_pred), rmse_rf]
})
display(resultaten)

,Model,R2,RMSE
0,Lineaire regressie,0.032454,20.287023
1,Random Forest,0.052000,20.081057
